<a href="https://colab.research.google.com/github/tau797/TelegramBot/blob/main/TelegramBotTAU_Sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Телеграм-бот, ведущий список задач***

In [5]:
!pip install telebot

In [35]:
# Телеграм-бот, ведущий список задач
# (C) TAU март 2024

#если нужно, устанавливаем библиотеку
#!pip install telebot

# Библиотеки
from random import choice
import telebot as tb
from telebot import types
import shelve # для сохранения состояния бота (этапа диалога)

# уникальный идентификатор бота - токен
token = '6886566042:AAGlHIz56IneBdd-G_QKHTmlYZ2fOxBSBOA'

#имя файла для хранения списка дел
FILE = "MyTODO.txt"

#моё имя Андрей
hoz = 'Андрей'

#задаем признак тега
tg = '#'

#Создание бота
bot = tb.TeleBot(token)

RANDOM_TASKS = ["Написать книгу #работа", "Написать Телеграм-бота #работа", "Починить машину #авто"]

todos = dict()

HELP = '''
Список доступных команд:
/покажи - напечать все задачи на заданную дату
например:
'/покажи сегодня' или '/покажь 29.03'

/все -  показать все имеющиеся задачи

/по - показать задачи выбранного вида
например: '/по #дом'

/еще - добавить задачу
например:
'/еще сегодня совершить подвиг'

Вы можете указать род задачи после тега '#', например
'/еще сегодня написать статью #работа'

/random - добавить на сегодня случайную задачу

/help - эта  подсказка

'''

#ПОРЯДОК ОБРАБОТЧИКОВ В ТЕКСТЕ ПРОГРАММЫ ВАЖЕН!!!


#При запуске
@bot.message_handler(commands=['start'])
def privet(message):
    markup = types.ReplyKeyboardMarkup()
    k1 = types.KeyboardButton('/помоги')
    k2 = types.KeyboardButton('/покажи сегодня')
    k3 = types.KeyboardButton('/все')
    markup.row(k1, k2, k3)
    bot.send_message(message.chat.id, f'<i>Приветствую, <b>{message.from_user.first_name}</b></i>',parse_mode='html',reply_markup=markup)
    bot.send_message(message.chat.id, "Чтобы узнать возможности, пиши /help\nКак Вас зовут?")
    if (hoz in message.text):
           bot.send_message(message.chat.id, 'Здравствуй, Хозяин!')

    chat_id = message.chat.id
    storage = shelve.open('shelve')
    storage[str(chat_id)] = 'zdorov'
    storage.close()

    #add_todo('сегодня', 'изучить бота TAU!') # шутка


# добавление задачи на дату
def add_todo(date, task):
    date = date.lower()
    if todos.get(date) is not None:
      if not task in todos[date]: #добавляем ежели не дубль
        todos[date].append(task)
      else:
        return f'Задача {task} уже была запланирована на {date}'
    else:
        todos[date] = [task]
    #и сохраняем перманентно
    try:
           file = open(FILE, 'wt')
           file.write(str(todos))
           file.close()
    except:
           print("Не могу дописать в файл!")
    return f'Задача {task} запланирована на {date}'

# добавляем реакцию бота на команду 'help'
@bot.message_handler(commands=['help','помоги'])
def help(message):
    bot.send_message(message.chat.id, HELP)

# добавляем реакцию бота на команду 'random'
@bot.message_handler(commands=['random'])
def random(message):
    task = choice(RANDOM_TASKS)
    bot.send_message(message.chat.id, add_todo('сегодня', task))

# добавляем реакцию бота на команду 'add'
@bot.message_handler(commands=['add','еще','ещё'])
def add(message):
    # обработка пустого задания
    if len(message.text)==4:
        bot.send_message(message.chat.id, 'нужно указать дату и задачу')
        return
    _, date, tail = message.text.split(maxsplit=2)
    #print(date,tail)

    task = ' '.join([tail])
    #print (task)
    add_todo(date, task)
    bot.send_message(message.chat.id, 'Задача добавлена!')

# добавляем реакцию бота на команду 'print'
@bot.message_handler(commands=['print','покажь','покажи'])
def print_(message):
    # обработка пустого задания
    if len(message.text)<10:
        bot.send_message(message.chat.id, 'нужно указать дату')
        return
    date = message.text.split()[1].lower()

    if date in todos:
        tasks = ''
        for task in todos[date]:
            #tasks += f'[ ] {task}\n'
            tasks += (task+'\n')
    else:
        tasks = 'На этот день ничего не найдено'
    bot.send_message(message.chat.id, tasks)

# добавляем реакцию бота на команду 'all'
@bot.message_handler(commands=['all','все','всё'])
def all_(message):
    #показываем все
    ll = " "
    for date,task in todos.items():
          ll += str(date)+': '+str(task)+'\n'
    bot.send_message(message.chat.id, ll)


# добавляем реакцию бота на команду 'show'
@bot.message_handler(commands=['show','по','вид'])
def show_(message):
    if len(message.text)==0:
        vid=' '
    else:
      if tg in message.text:
        ll = '-'
        vid = message.text.split(tg)[1].lower()

        for date,task in todos.items():
          for el in task:
            #print(task) отладочная печать
            if (vid in str(el)):
              ll += str(date)+': '+str(el)+'\n'
      else:
        ll = 'Укажите вид задачи после '+tg
    bot.send_message(message.chat.id, ll)

# @bot.message_handler(content_types=["text"])
# def text(mess):
#         bot.send_message(mess.chat.id, '>')
#         if (hoz in mess.text):
#            bot.send_message(mess.chat.id, 'Здравствуй, Хозяин!')

@bot.message_handler(func=lambda mess: mess.content_type == 'text')
def zdorov(mess):
    chat_id = mess.chat.id
    storage = shelve.open('shelve')
    state = storage[str(chat_id)]
    if state != 'zdorov':
        pass
        storage.close()
    else:
        #storage[str(chat_id)] = 'y'
        storage.close()
        #bot.send_message(mess.chat.id, '>')
        if (hoz in mess.text):
           bot.send_message(mess.chat.id, 'Здравствуй, Хозяин!')

bot.polling(none_stop=True)


2024-04-04 00:16:45,417 (__init__.py:1147 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message must be non-empty"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message must be non-empty
2024-04-04 00:16:45,421 (__init__.py:1149 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1141, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 149, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 92, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 7868, in _run_middlewares_and_handler
    r